In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
#python3 -m spacy download en

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\John\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [4]:
#pip install spacy==2.2.0

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\John\anaconda3\envs\hackMT\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [6]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
#import data
df_id = pd.read_csv("dataset/nonprofit.txt", sep = "|", encoding = "cp1252")
df_text = pd.read_csv("dataset/nonprofit_text.txt", sep = "|", encoding = "cp1252")

In [8]:
#create sample set
df_text = df_text.sample(n = 250000)
df_text.head()

,nonprofit_text_id,reporting_year,nonprofit_id,grouptype,description
1116312,1118265,2018,358154,businessname,Rutherford County Emergency Food Bank Inc
1761144,1764508,2020,246984,missiondesc,"VOLUNTEER FIRE DEPARTMENT, WHICH PROVIDES FIRE..."
707578,708713,2020,603049,desc,Business Membership Organization
509907,510808,2017,59673,desc,THE ROAD AT WEST GENESEE DISTRIBUTIONS FROM RE...
1639239,1642528,2020,82475,missiondesc,Construct trails for public use in snowmobilin...


In [9]:
data = df_text.description.values.tolist()
data

['Rutherford County Emergency Food Bank Inc',
 'VOLUNTEER FIRE DEPARTMENT, WHICH PROVIDES FIRE AND EMERGENCY MEDICAL SERVICES TO THE COMMUNITIES THAT ARE SERVED WITHIN THEIR JURISDICTION.',
 'Business Membership Organization',
 'THE ROAD AT WEST GENESEE DISTRIBUTIONS FROM RESTRICTED FUNDS',
 'Construct trails for public use in snowmobiling, to promote safe snowmobiling, to provide education for snowmobilers, and to provide assistance in enforcing safe snowmobiling.',
 "THE HOMELESS SERVICES NETWORK OF CENTRAL FLORIDA IS THE LEAD AGENCY FOR THE HUD CONTINUUM OF CARE SUPPORTIVE HOUSING PROGRAM, SHELTER, AND CARE FUNDING FOR OSCEOLA, SEMINOLE, AND ORANGE COUNTIES INCLUDING THE CITY OF ORLANDO. IT ADMINISTERS MORE THAN 40 GRANTS THROUGH ITS SUB-RECIPIENT AGENCIES WHO PROVIDE SERVICES TO THE HOMELESS IN ORANGE, SEMINOLE AND OSCEOLA COUNTIES. IT ALSO FUNDS PROGRAMS THAT PROVIDE TRANSITIONAL AND PERMANENT HOUSING, SUPPORTIVE SERVICES, AND ONGOING CASE MANAGEMENT FOR THE REGION'S HOMELESS, AND

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence).encode('utf-8'), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
print(data_words[:1])

[['rutherford', 'county', 'emergency', 'food', 'bank', 'inc']]


In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['rutherford', 'county', 'emergency', 'food_bank', 'inc']


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
#python3 -m spacy download en

In [18]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

print(data_words_nostops[:1])

[['rutherford', 'county', 'emergency', 'food', 'bank', 'inc']]


In [19]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

print(data_words_bigrams[:1])

[['rutherford', 'county', 'emergency', 'food', 'bank', 'inc']]


In [31]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)

print(data_lemmatized[:1])


[[]]


In [ ]:
print(data_lemmatized[:2])

In [23]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:2])

[[], [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]]


In [28]:
# Build LDA model
lda_model = gensim.models.ldamulticore.LdaMulticore(workers = 3, corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=42,
                                           chunksize=100,
                                           passes=10,
                                           per_word_topics=True)

In [29]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.052*"provide" + 0.047*"service" + 0.028*"care" + 0.027*"family" + '
  '0.023*"child" + 0.021*"health" + 0.016*"need" + 0.016*"community" + '
  '0.015*"support" + 0.014*"program"'),
 (1,
  '0.046*"provide" + 0.033*"organization" + 0.031*"support" + 0.026*"member" + '
  '0.025*"fund" + 0.023*"purpose" + 0.020*"benefit" + 0.018*"activity" + '
  '0.016*"scholarship" + 0.016*"charitable"'),
 (2,
  '0.027*"community" + 0.021*"promote" + 0.013*"member" + 0.012*"education" + '
  '0.010*"development" + 0.010*"business" + 0.010*"work" + 0.009*"public" + '
  '0.009*"people" + 0.008*"support"'),
 (3,
  '0.045*"program" + 0.038*"school" + 0.034*"student" + 0.031*"provide" + '
  '0.017*"child" + 0.017*"youth" + 0.015*"education" + 0.014*"low" + '
  '0.012*"high" + 0.011*"year"'),
 (4,
  '0.016*"art" + 0.014*"project" + 0.013*"public" + 0.013*"research" + '
  '0.012*"new" + 0.011*"water" + 0.009*"include" + 0.008*"preserve" + '
  '0.007*"history" + 0.006*"artist"')]


In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.338095924961603

Coherence Score:  0.5826799990458551


In [32]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

AttributeError: module 'pyLDAvis' has no attribute 'gensim'